<a href="https://colab.research.google.com/github/wendyhv/Cancer-Prediction-using-Convolutional-Neural-Network/blob/main/Cancer_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import cv2
from google.colab.patches import cv2_imshow
from skimage import io

# Install pydicom package, which is used for handling DICOM files
!pip install pydicom
import pydicom
import glob  # Used for file locations
from PIL import Image  # Used for image processing
from skimage.transform import resize  # Used for resizing images
import copy  # Used for creating copies of objects
import matplotlib.pyplot as plt  # Used for plotting

In [ ]:
# Import the drive module from the google.colab package
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Set IPython to display all outputs from a cell (not just the last one)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

# New section

In [ ]:
# variable image path
def image_processing_function(image_folder_path):
    i=0
    image_extension= "*.dcm"
    image_files = glob.glob(image_folder_path+image_extension)
    images = np.empty((len(image_files),512,512),dtype=np.int8)
    for file in image_files:
      dcm = pydicom.dcmread(file)
      img = dcm.pixel_array.astype(np.int8)
      if img.shape !=(512, 512):
        print(dcm.filename)
      max_value = img.max()
      min_value = img.min()
      if max_value != min_value:
        img = (img - min_value) / ((max_value - min_value)*255)
      else:
        img = img * 0.0
      images = np.append(images, img[np.newaxis,...],axis=0)
      #images[i] = np.array(img)
     # i+=1
      #images[i] = np.stack([images,np.array(img)],axis=0)
      #images = np.append(images, np.array(img))
      #images = np.array(images)

    # Convert the list of images into a numpy array
    return images

TCGA-UCEC

In [ ]:
image_path1 ="/content/drive/MyDrive/TCGA-UCEC/TCGA-D1-A174/"
image_path2 ="/content/drive/MyDrive/TCGA-UCEC/TCGA-D1-A1NZ/"
image_path3 ="/content/drive/MyDrive/TCGA-UCEC/TCGA-FI-A2F4/"
image_path4 ="/content/drive/MyDrive/TCGA-UCEC/TCGA-D1-A160/"
image_path5 ="/content/drive/MyDrive/TCGA-UCEC/TCGA-FI-A2D5/"
image_path6 ="/content/drive/MyDrive/TCGA-UCEC/Non_genes/"

TCGA_D1_A174_images= image_processing_function(image_path1)
TCGA_D1_A1NZ_images= image_processing_function(image_path2)
TCGA_FI_A2F4_images= image_processing_function(image_path3)
TCGA_D1_A160_images= image_processing_function(image_path4)
TCGA_FI_A2D5_images= image_processing_function(image_path5)
TCGA_Non_genes_images= image_processing_function(image_path6)

<ipython-input-4-c3dd485babe1>:15: RuntimeWarning: overflow encountered in byte_scalars
  img = (img - min_value) / ((max_value - min_value)*255)


In [ ]:
TCGA_D1_A174_images.shape
TCGA_D1_A1NZ_images.shape
TCGA_FI_A2F4_images.shape
TCGA_D1_A160_images.shape
TCGA_FI_A2D5_images.shape
TCGA_Non_genes_images.shape

(106, 512, 512)

(108, 512, 512)

(600, 512, 512)

In [ ]:
label_TCGA_D1_A174_images_LATS1 = np.full(TCGA_D1_A174_images.shape[0], 1) # Lats1
label_TCGA_D1_A1NZ_images_LATS2 = np.full(TCGA_D1_A1NZ_images.shape[0], 2) # Lats2
label_TCGA_FI_A2F4_images_MST1 = np.full(TCGA_FI_A2F4_images.shape[0], 3) # Mst1
label_TCGA_D1_A160_images_YAP1 = np.full(TCGA_D1_A160_images.shape[0], 4) # Yap1
label_TCGA_FI_A2D5_images_TAZ = np.full(TCGA_FI_A2D5_images.shape[0], 5)   # Taz
label_TCGA_Non_genes_images = np.full(TCGA_Non_genes_images.shape[0], 0) # Non genes

In [ ]:
merged_labels_LATS1 = np.concatenate((label_TCGA_D1_A174_images_LATS1,label_TCGA_Non_genes_images))
merged_image_LATS1 = np.concatenate((TCGA_D1_A174_images,TCGA_Non_genes_images), axis=0)
merged_labels_LATS2 = np.concatenate((label_TCGA_D1_A1NZ_images_LATS2,label_TCGA_Non_genes_images))
merged_image_LATS2 = np.concatenate((TCGA_D1_A1NZ_images,TCGA_Non_genes_images), axis=0)
merged_labels_MST1 = np.concatenate((label_TCGA_FI_A2F4_images_MST1,label_TCGA_Non_genes_images))
merged_image_MST1 = np.concatenate((TCGA_FI_A2F4_images,TCGA_Non_genes_images), axis=0)
merged_labels_YAP1 = np.concatenate((label_TCGA_D1_A160_images_YAP1,label_TCGA_Non_genes_images))
merged_image_YAP1 = np.concatenate((TCGA_D1_A160_images,TCGA_Non_genes_images), axis=0)
merged_labels_TAZ = np.concatenate((label_TCGA_FI_A2D5_images_TAZ,label_TCGA_Non_genes_images))
merged_image_TAZ = np.concatenate((TCGA_FI_A2D5_images,TCGA_Non_genes_images), axis=0)

In [ ]:
merged_image_LATS1.shape
merged_image_LATS2.shape
merged_image_MST1.shape
merged_image_YAP1.shape
merged_image_TAZ.shape

(706, 512, 512)

(708, 512, 512)

TCGA-STAD

In [ ]:
# image_path1 ="/content/drive/MyDrive/TCGA-STAD/TCGA-VQ-AA6K/"
# image_path2 ="/content/drive/MyDrive/TCGA-STAD/TCGA-VQ-A8E7/"
# image_path3 ="/content/drive/MyDrive/TCGA-STAD/TCGA-VQ-A91D/"
# image_path4 ="/content/drive/MyDrive/TCGA-STAD/TCGA-VQ-A8P2/"
# image_path5 ="/content/drive/MyDrive/TCGA-STAD/Non_genes/"

# TCGA_VQ_AA6K_images= image_processing_function(image_path1)
# TCGA_VQ_A8E7_images= image_processing_function(image_path2)
# TCGA_VQ_A91D_images= image_processing_function(image_path3)
# TCGA_VQ_A8P2_images= image_processing_function(image_path4)
# TCGA_Non_genes_images= image_processing_function(image_path5)

<ipython-input-4-c3dd485babe1>:15: RuntimeWarning: overflow encountered in byte_scalars
  img = (img - min_value) / ((max_value - min_value)*255)


In [ ]:
# TCGA_VQ_AA6K_images.shape
# TCGA_VQ_A8E7_images.shape
# TCGA_VQ_A91D_images.shape
# TCGA_VQ_A8P2_images.shape
# TCGA_Non_genes_images.shape

(108, 512, 512)

(104, 512, 512)

(106, 512, 512)

(102, 512, 512)

(600, 512, 512)

In [ ]:
# label_TCGA_VQ_AA6K_images_LATS1 = np.full(TCGA_VQ_AA6K_images.shape[0], 1) # Lats1
# label_TCGA_VQ_A8E7_images_LATS2 = np.full(TCGA_VQ_A8E7_images.shape[0], 2) # Lats2
# label_TCGA_VQ_A91D_images_MST1 = np.full(TCGA_VQ_A91D_images.shape[0], 3) # Mst1
# label_TCGA_VQ_A8P2_images_YAP1 = np.full(TCGA_VQ_A8P2_images.shape[0], 4) # Yap1
# label_TCGA_Non_genes_images = np.full(TCGA_Non_genes_images.shape[0], 0) # Non genes

In [ ]:
# merged_labels_LATS1 = np.concatenate((label_TCGA_VQ_AA6K_images_LATS1,label_TCGA_Non_genes_images))
# merged_image_LATS1 = np.concatenate((TCGA_VQ_AA6K_images,TCGA_Non_genes_images), axis=0)

In [ ]:
# merged_image_LATS1.shape

(708, 512, 512)

TCGA-BLCA

In [ ]:
# image_path1 ="/content/drive/MyDrive/TCGA-BLCA/TCGA-XF-AAN2/"
# image_path2 ="/content/drive/MyDrive/TCGA-BLCA/TCGA-ZF-A9RD/"
# image_path3 ="/content/drive/MyDrive/TCGA-BLCA/TCGA-XF-AAN0/"
# image_path4 ="/content/drive/MyDrive/TCGA-BLCA/TCGA-FD-A3SO/"
# image_path5 ="/content/drive/MyDrive/TCGA-BLCA/TCGA-XF-A9T3/"
# image_path6 ="/content/drive/MyDrive/TCGA-BLCA/Non_genes/"

# TCGA_XF_AAN2_images= image_processing_function(image_path1)
# TCGA_ZF_A9RD_images= image_processing_function(image_path2)
# TCGA_XF_AAN0_images= image_processing_function(image_path3)
# TCGA_FD_A3SO_images= image_processing_function(image_path4)
# TCGA_XF_A9T3_images= image_processing_function(image_path5)
# TCGA_Non_genes_images= image_processing_function(image_path6)

<ipython-input-4-c3dd485babe1>:15: RuntimeWarning: overflow encountered in byte_scalars
  img = (img - min_value) / ((max_value - min_value)*255)


In [ ]:
# TCGA_XF_AAN2_images.shape
# TCGA_ZF_A9RD_images.shape
# TCGA_XF_AAN0_images.shape
# TCGA_FD_A3SO_images.shape
# TCGA_XF_A9T3_images.shape
# TCGA_Non_genes_images.shape

(100, 512, 512)

(60, 512, 512)

(104, 512, 512)

(106, 512, 512)

(102, 512, 512)

(600, 512, 512)

In [ ]:
# label_TCGA_XF_AAN2_images_LATS1 = np.full(TCGA_XF_AAN2_images.shape[0], 1) # Lats1
# label_TCGA_ZF_A9RD_images_LATS2 = np.full(TCGA_ZF_A9RD_images.shape[0], 2) # Lats2
# label_TCGA_XF_AAN0_images_MST1 = np.full(TCGA_XF_AAN0_images.shape[0], 3) # Mst1
# label_TCGA_FD_A3SO_images_YAP1 = np.full(TCGA_FD_A3SO_images.shape[0], 4) # Yap1
# label_TCGA_XF_A9T3_images_TAZ = np.full(TCGA_XF_A9T3_images.shape[0], 5)   # Taz
# label_TCGA_Non_genes_images = np.full(TCGA_Non_genes_images.shape[0], 0) # Non genes

In [ ]:
# merged_labels_LATS1 = np.concatenate((label_TCGA_XF_AAN2_images_LATS1,label_TCGA_Non_genes_images))
# merged_image_LATS1 = np.concatenate((TCGA_XF_AAN2_images,TCGA_Non_genes_images), axis=0)

In [ ]:
# merged_image_LATS1.shape

(700, 512, 512)

TCGA-OV

In [ ]:
# image_path1 ="/content/drive/MyDrive/TCGA-OV/TCGA-30-1891/"
# image_path6 ="/content/drive/MyDrive/TCGA-OV/Non_genes/"

# TCGA_30_1891_images= image_processing_function(image_path1)
# TCGA_Non_genes_images= image_processing_function(image_path6)

<ipython-input-4-c3dd485babe1>:15: RuntimeWarning: overflow encountered in byte_scalars
  img = (img - min_value) / ((max_value - min_value)*255)


In [ ]:
# TCGA_30_1891_images.shape
# TCGA_Non_genes_images.shape

(108, 512, 512)

(594, 512, 512)

In [ ]:
# label_TCGA_30_1891_images_LATS1 = np.full(TCGA_30_1891_images.shape[0], 1) # Lats1
# label_TCGA_Non_genes_images = np.full(TCGA_Non_genes_images.shape[0], 0) # Non genes

In [ ]:
# merged_labels_LATS1 = np.concatenate((label_TCGA_30_1891_images_LATS1,label_TCGA_Non_genes_images))
# merged_image_LATS1 = np.concatenate((TCGA_30_1891_images,TCGA_Non_genes_images), axis=0)

In [ ]:
# merged_image_LATS1.shape

(702, 512, 512)

TCGA-LUAD

In [ ]:
# image_path1 ="/content/drive/MyDrive/TCGA-LUAD/TCGA-17-Z031/"
# image_path2 ="/content/drive/MyDrive/TCGA-LUAD/TCGA-50-5044/"
# image_path4 ="/content/drive/MyDrive/TCGA-LUAD/TCGA-50-5049/"
# image_path6 ="/content/drive/MyDrive/TCGA-LUAD/Non_genes/"

# TCGA_17_Z031_images= image_processing_function(image_path1)
# TCGA_50_5044_images= image_processing_function(image_path2)
# TCGA_50_5049_images= image_processing_function(image_path4)
# TCGA_Non_genes_images= image_processing_function(image_path6)

<ipython-input-4-c3dd485babe1>:15: RuntimeWarning: overflow encountered in byte_scalars
  img = (img - min_value) / ((max_value - min_value)*255)


In [ ]:
# TCGA_17_Z031_images.shape
# TCGA_50_5044_images.shape
# TCGA_50_5049_images.shape
# TCGA_Non_genes_images.shape

(106, 512, 512)

(86, 512, 512)

(108, 512, 512)

(582, 512, 512)

In [ ]:
# label_TCGA_17_Z031_images_LATS1 = np.full(TCGA_17_Z031_images.shape[0], 1) # Lats1
# label_TCGA_50_5044_images_LATS2 = np.full(TCGA_50_5044_images.shape[0], 2) # Lats2
# label_TCGA_50_5049_images_YAP1 = np.full(TCGA_50_5049_images.shape[0], 4) # YAP1
# label_TCGA_Non_genes_images = np.full(TCGA_Non_genes_images.shape[0], 0) # Non genes

In [ ]:
# merged_labels_LATS1 = np.concatenate((label_TCGA_17_Z031_images_LATS1,label_TCGA_Non_genes_images))
# merged_image_LATS1 = np.concatenate((TCGA_17_Z031_images,TCGA_Non_genes_images), axis=0)

In [ ]:
# merged_image_LATS1.shape

(688, 512, 512)

TCGA-UCEC vs Other Cancers

In [ ]:
# image_path1 ="/content/drive/MyDrive/ LATS1-UCEC/"
# image_path6 ="/content/drive/MyDrive/LATS1-Other cancers/"

# TCGA_LATS1_UCEC_images= image_processing_function(image_path1)
# TCGA_LATS1_Other_Cancers_images= image_processing_function(image_path6)

<ipython-input-4-c3dd485babe1>:15: RuntimeWarning: overflow encountered in byte_scalars
  img = (img - min_value) / ((max_value - min_value)*255)


In [ ]:
# TCGA_LATS1_UCEC_images.shape
# TCGA_LATS1_Other_Cancers_images.shape

(300, 512, 512)

(688, 512, 512)

In [ ]:
# label_TCGA_LATS1_UCEC_images = np.full(TCGA_LATS1_UCEC_images.shape[0], 1) # LATS1 UCEC
# label_TCGA_LATS1_Other_Cancers_images = np.full(TCGA_LATS1_Other_Cancers_images.shape[0], 0) # LATS1 Other cancers

In [ ]:
# merged_labels_LATS1 = np.concatenate((label_TCGA_LATS1_UCEC_images,label_TCGA_LATS1_Other_Cancers_images))
# merged_image_LATS1 = np.concatenate((TCGA_LATS1_UCEC_images,TCGA_LATS1_Other_Cancers_images), axis=0)

In [ ]:
# merged_image_LATS1.shape

(988, 512, 512)

CNN model

In [ ]:
# Assuming your input images are stored in an array called 'images'
batch_size = merged_image_LATS1.shape[0]
height = merged_image_LATS1.shape[1]
width = merged_image_LATS1.shape[2]

# Add a channel dimension with size 1
merged_image_LATS1 = np.reshape(merged_image_LATS1, (batch_size, height, width, 1))

In [ ]:
# Import machine learning libraries
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Load the labeled images and labels arrays

# Split the data into training and testing sets
X1_train, X1_test, y1_train, y1_test = train_test_split(merged_image_LATS1, merged_labels_LATS1, test_size=0.2, random_state=42)

# Define the CNN model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=X1_train.shape[1:]))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model on the training data
model.fit(X1_train, y1_train, epochs=4, validation_data=(X1_test, y1_test))

loss, accuracy = model.evaluate(X1_test, y1_test)

# Print the test accuracy
print('Test accuracy:', accuracy)

Epoch 1/4
18/18 [==============================] - 24s 462ms/step - loss: 0.5512 - accuracy: 0.8830 - val_loss: 0.3714 - val_accuracy: 0.8732
Epoch 2/4
18/18 [==============================] - 4s 214ms/step - loss: 0.2484 - accuracy: 0.9202 - val_loss: 0.2091 - val_accuracy: 0.9296
Epoch 3/4
18/18 [==============================] - 4s 213ms/step - loss: 0.2244 - accuracy: 0.9238 - val_loss: 0.2089 - val_accuracy: 0.9296
Epoch 4/4
18/18 [==============================] - 4s 213ms/step - loss: 0.2228 - accuracy: 0.9238 - val_loss: 0.2316 - val_accuracy: 0.9296


5/5 [==============================] - 0s 54ms/step - loss: 0.2316 - accuracy: 0.9296
Test accuracy: 0.9295774698257446


Chi-Square test

In [ ]:
# Assuming your input images are stored in an array called 'images'
batch_size1 = merged_image_LATS1.shape[0]
height1 = merged_image_LATS1.shape[1]
width1 = merged_image_LATS1.shape[2]
batch_size2 = merged_image_LATS2.shape[0]
height2 = merged_image_LATS2.shape[1]
width2 = merged_image_LATS2.shape[2]

# Add a channel dimension with size 1
merged_image_LATS1 = np.reshape(merged_image_LATS1, (batch_size1, height1, width1, 1))
merged_image_LATS2 = np.reshape(merged_image_LATS2, (batch_size2, height2, width2, 1))

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Load the labeled images and labels arrays

# Split the data into training and testing sets
X1_train, X1_test, y1_train, y1_test = train_test_split(merged_image_LATS1, merged_labels_LATS1, test_size=0.2, random_state=42)
X2_train, X2_test, y2_train, y2_test = train_test_split(merged_image_LATS2, merged_labels_LATS2, test_size=0.2, random_state=42)

# Define the CNN model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=X1_train.shape[1:]))
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=X2_train.shape[1:]))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from scipy.stats import chi2_contingency # Used for statistical testing in a contingency table

# Convert images to categorical data
X1_categorical = np.full(X1_test.shape[0], 1)
X2_categorical = np.full(X2_test.shape[0], 0)

# Create a contingency table
contingency_table = np.array([
    [np.sum(X1_categorical == 1), np.sum(X2_categorical == 1)],
    [np.sum(X1_categorical == 0), np.sum(X2_categorical == 0)]
])

# Perform chi-square test
chi2, p, _, _ = chi2_contingency(contingency_table)

print("Chi-Square Value:", chi2)
print("P-Value:", p)

Chi-Square Value: 280.01408450704224
P-Value: 7.456360215413788e-63


T test

In [ ]:
# from scipy import stats

TCGA-UCEC

In [ ]:
# Perform the two-sample t-test
# t_statistic, p_value = stats.ttest_ind(predicted_classes_LATS1, predicted_classes_LATS2)

# Print the results
# print("t-statistic:", t_statistic)
# print("p-value:", p_value)

t-statistic: -0.20454770253238203
p-value: 0.8380730695517762


In [ ]:
# Assuming your input images are stored in an array called 'images'
# batch_size1 = merged_image_LATS1.shape[0]
# height1 = merged_image_LATS1.shape[1]
# width1 = merged_image_LATS1.shape[2]
# batch_size2 = merged_image_LATS2.shape[0]
# height2 = merged_image_LATS2.shape[1]
# width2 = merged_image_LATS2.shape[2]
# batch_size3 = merged_image_MST1.shape[0]
# height3 = merged_image_MST1.shape[1]
# width3 = merged_image_MST1.shape[2]
# batch_size4 = merged_image_YAP1.shape[0]
# height4 = merged_image_YAP1.shape[1]
# width4 = merged_image_YAP1.shape[2]
# batch_size5 = merged_image_TAZ.shape[0]
# height5 = merged_image_TAZ.shape[1]
# width5 = merged_image_TAZ.shape[2]

# Add a channel dimension with size 1
# merged_image_LATS1 = np.reshape(merged_image_LATS1, (batch_size1, height1, width1, 1))
# merged_image_LATS2 = np.reshape(merged_image_LATS2, (batch_size2, height2, width2, 1))
# merged_image_MST1 = np.reshape(merged_image_MST1, (batch_size3, height3, width3, 1))
# merged_image_YAP1 = np.reshape(merged_image_YAP1, (batch_size4, height4, width4, 1))
# merged_image_TAZ = np.reshape(merged_image_TAZ, (batch_size5, height5, width5, 1))

In [ ]:
# from sklearn.model_selection import train_test_split
# from keras.models import Sequential
# from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Load the labeled images and labels arrays

# Split the data into training and testing sets
# X1_train, X1_test, y1_train, y1_test = train_test_split(merged_image_LATS1, merged_labels_LATS1, test_size=0.2, random_state=42)
# X2_train, X2_test, y2_train, y2_test = train_test_split(merged_image_LATS2, merged_labels_LATS2, test_size=0.2, random_state=42)
# X3_train, X3_test, y3_train, y3_test = train_test_split(merged_image_MST1, merged_labels_MST1, test_size=0.2, random_state=42)
# X4_train, X4_test, y4_train, y4_test = train_test_split(merged_image_YAP1, merged_labels_YAP1, test_size=0.2, random_state=42)
# X5_train, X5_test, y5_train, y5_test = train_test_split(merged_image_TAZ, merged_labels_TAZ, test_size=0.2, random_state=42)

# Define the CNN model architecture
# model = Sequential()
# model.add(Conv2D(32, (3, 3), activation='relu', input_shape=X1_train.shape[1:]))
# model.add(Conv2D(32, (3, 3), activation='relu', input_shape=X2_train.shape[1:]))
# model.add(Conv2D(32, (3, 3), activation='relu', input_shape=X3_train.shape[1:]))
# model.add(Conv2D(32, (3, 3), activation='relu', input_shape=X4_train.shape[1:]))
# model.add(Conv2D(32, (3, 3), activation='relu', input_shape=X5_train.shape[1:]))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Flatten())
# model.add(Dense(64, activation='relu'))
# model.add(Dense(3, activation='softmax'))

# Compile the model
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model on the training data of each group
# model.fit(X1_train, y1_train, epochs=4, validation_data=(X1_test, y1_test))
# model.fit(X2_train, y2_train, epochs=4, validation_data=(X2_test, y2_test))
# model.fit(X3_train, y3_train, epochs=4, validation_data=(X3_test, y3_test))
# model.fit(X4_train, y4_train, epochs=4, validation_data=(X4_test, y4_test))
# model.fit(X5_train, y5_train, epochs=4, validation_data=(X5_test, y5_test))

Epoch 1/4
18/18 [==============================] - 78s 2s/step - loss: 0.5810 - accuracy: 0.8369 - val_loss: 0.5025 - val_accuracy: 0.8028
Epoch 2/4
18/18 [==============================] - 17s 920ms/step - loss: 0.4309 - accuracy: 0.8617 - val_loss: 0.5024 - val_accuracy: 0.8028
Epoch 3/4
18/18 [==============================] - 17s 936ms/step - loss: 0.3888 - accuracy: 0.8617 - val_loss: 0.3904 - val_accuracy: 0.8028
Epoch 4/4
18/18 [==============================] - 17s 939ms/step - loss: 0.2262 - accuracy: 0.9184 - val_loss: 0.2166 - val_accuracy: 0.9296


Epoch 1/4
18/18 [==============================] - 36s 2s/step - loss: 12.6380 - accuracy: 0.8481 - val_loss: 0.5295 - val_accuracy: 0.8099
Epoch 2/4
18/18 [==============================] - 17s 952ms/step - loss: 0.3643 - accuracy: 0.8746 - val_loss: 0.4253 - val_accuracy: 0.8592
Epoch 3/4
18/18 [==============================] - 17s 953ms/step - loss: 0.2257 - accuracy: 0.9170 - val_loss: 0.1949 - val_accuracy: 0.9366
Epoch 4/4
18/18 [==============================] - 17s 941ms/step - loss: 0.2238 - accuracy: 0.9205 - val_loss: 0.1964 - val_accuracy: 0.9366


Epoch 1/4
18/18 [==============================] - 37s 2s/step - loss: nan - accuracy: 0.8663 - val_loss: nan - val_accuracy: 0.8085
Epoch 2/4
18/18 [==============================] - 16s 910ms/step - loss: nan - accuracy: 0.8663 - val_loss: nan - val_accuracy: 0.8085
Epoch 3/4
18/18 [==============================] - 16s 912ms/step - loss: nan - accuracy: 0.8663 - val_loss: nan - val_accuracy: 0.8085
Epoch 4/4
18/18 [==============================] - 16s 916ms/step - loss: nan - accuracy: 0.8663 - val_loss: nan - val_accuracy: 0.8085


Epoch 1/4
18/18 [==============================] - 34s 2s/step - loss: nan - accuracy: 0.8632 - val_loss: nan - val_accuracy: 0.8085
Epoch 2/4
18/18 [==============================] - 17s 925ms/step - loss: nan - accuracy: 0.8632 - val_loss: nan - val_accuracy: 0.8085
Epoch 3/4
18/18 [==============================] - 17s 925ms/step - loss: nan - accuracy: 0.8632 - val_loss: nan - val_accuracy: 0.8085
Epoch 4/4
18/18 [==============================] - 17s 925ms/step - loss: nan - accuracy: 0.8632 - val_loss: nan - val_accuracy: 0.8085


Epoch 1/4
18/18 [==============================] - 36s 2s/step - loss: nan - accuracy: 0.8661 - val_loss: nan - val_accuracy: 0.8214
Epoch 2/4
18/18 [==============================] - 17s 923ms/step - loss: nan - accuracy: 0.8661 - val_loss: nan - val_accuracy: 0.8214
Epoch 3/4
18/18 [==============================] - 17s 923ms/step - loss: nan - accuracy: 0.8661 - val_loss: nan - val_accuracy: 0.8214
Epoch 4/4
18/18 [==============================] - 17s 925ms/step - loss: nan - accuracy: 0.8661 - val_loss: nan - val_accuracy: 0.8214


In [ ]:
# Evaluate the model on the testing data of each group
# loss_LATS1, accuracy_LATS1 = model.evaluate(X1_test, y1_test)
# loss_LATS2, accuracy_LATS2 = model.evaluate(X2_test, y2_test)
# loss_MST1, accuracy_MST1 = model.evaluate(X3_test, y3_test)
# loss_YAP1, accuracy_YAP1 = model.evaluate(X4_test, y4_test)
# loss_TAZ, accuracy_TAZ = model.evaluate(X5_test, y5_test)

5/5 [==============================] - 1s 165ms/step - loss: nan - accuracy: 0.8214


Anova

In [ ]:
# from scipy.stats import f_oneway

TCGA-UCEC

In [ ]:
# Wrap scalar values in lists
# evaluation_metrics_LATS1 = [accuracy_LATS1]
# evaluation_metrics_LATS2 = [accuracy_LATS2]
# evaluation_metrics_MST1 = [accuracy_MST1]
# evaluation_metrics_YAP1 = [accuracy_YAP1]
# evaluation_metrics_TAZ = [accuracy_TAZ]

In [ ]:
# Perform ANOVA to compare the performance among the groups
# f_statistic, p_value = f_oneway(
    evaluation_metrics_LATS1,
    evaluation_metrics_LATS2,
    evaluation_metrics_MST1,
    evaluation_metrics_YAP1,
    evaluation_metrics_TAZ
)

# Print the results
# print("F-statistic:", f_statistic)
# print("p-value:", p_value)

F-statistic: nan
p-value: nan
